In [ ]:
from jetbot import ObjectDetector  # for ObjectDetect, import the 'ObjectDetector' function from jetbot module

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')  #
#https://github.com/NVIDIA-AI-IOT/jetbot/wiki/examples

In [ ]:
from jetbot import Camera   # import the 'Camera' function from jetbot module

camera = Camera.instance(width=300, height=300)  #instance함수 사용

In [ ]:
import sys  # import the sys, 'sys' is System-specific parameters and functions. This module provides access to some variables used or maintained by the interpreter and to functions that interact strongly with the interpreter.
sys.path.append('/home/jetbot/snowboy/examples/Python3') # add the path 
import snowboydecoder #add snowboydecoder model? for using of snowboy
import signal
from jetbot import Robot
import time
from pydub import AudioSegment  # Jetson nano에서 음악을 재생하기 위한 module
from pydub.playback import play
import FaBo9Axis_MPU9250 # IMU sensor사용을 위한 import
#https://brisbaneroboticsclub.id.au/install-keystudio-i2c-shield-v1/
import numpy as np

# Demo code for listening to four hotwords at the same time

robot = Robot()
interrupted = False

def signal_handler(signal, frame):
    global interrupted
    interrupted = True


def interrupt_callback():
    global interrupted # 이부분 잘모르겠어요 loop때문에 넣은 건가요?
    return interrupted


models = ['howmuch_zzan.pmdl']

def howmuch(i): # 몇짠이냐고 물어본다. 
    if(i==1):
        song = AudioSegment.from_wav("1zzan.wav") # 1잔을 마셨으면(i==1) "1zzan.wav"파일을 재생한다. 
        play(song)
        global interrupted
        interrupted=True
    elif(i==2):
        song = AudioSegment.from_wav("2zzan.wav") # 2잔을 마셨으면(i==2) "2zzan.wav"파일을 재생한다. 
        play(song)
        interrupted=True
    elif(i==3):
        song = AudioSegment.from_wav("3zzan.wav") # 3잔을 마셨으면(i==3) "3zzan.wav"파일을 재생한다. 
        play(song)
        interrupted=True
    elif(i==4):
        song = AudioSegment.from_wav("4zzan.wav") # 4잔을 마셨으면(i==4) "4zzan.wav"파일을 재생한다.
        play(song)
        interrupted=True
    else:
        print("stop drinking!")  # 사용자가 5잔 이상을 마시려고 한다면 "stop drinking!"을 출력한다 (근데 )
        

# capture SIGINT signal, e.g., Ctrl+c
signal.signal(signal.SIGINT, signal_handler) 
#The bigger the value, the more senstive the decoder. If an empty list is provided, then the default sensitivity in the model will be used.
sensitivity = [0.5]
detector = snowboydecoder.HotwordDetector(models, sensitivity=sensitivity)
# In order to work properly, you must match the input and your logic sequence in the command.
callbacks = [lambda: howmuch(i)] 

i=1
while(i!=6):
    if(i<5):
        robot = Robot() #robot라는 객체를 만든다. 
        detections = model(camera.value) #카메라 정보를 담을 변수를 만든다. 
        #print(detections)
        image_number = 0
        object_number = 0
    
        if((detections[image_number][object_number]['label']==1)|(detections[image_number][object_number]['label']==47)|(detections[image_number][object_number]['label']==67)): #사람이 감지되는 상황
            robot.forward(0.2) # 사람이 감지되면 2초 앞으로 움직이고 멈춘다
            time.sleep(1.0) 
            robot.stop()
    
            mpu9250 = FaBo9Axis_MPU9250.MPU9250()  # IMU sensor사용을 위해 객체를 만든다. 
            accel=mpu9250.readAccel() # IMU sensor에서 발생하는 sensor값을 읽는다
            while((abs(accel['x'])<0.2)|(abs(accel['z'])<0.2)): # 가속도 값이 커지는 이벤트가 발생하기 전까지는 반복해서 accel value를 받아서 큰지 아닌지 검사한다.
                accel=mpu9250.readAccel()
        
            print('Cheers!') # 가속도 값이 커지는 이벤트가 발생하면 'cheers'가 발생한것이다.
            robot.backward(0.2)  # 그렇다면, while문을 벗어나서 '뒤로 가는'동작을 수행한다.
            time.sleep(1.0)
            robot.stop()
                    
            print('Listening... Press Ctrl+C to exit')
            detector.start(detected_callback=callbacks, interrupt_check=interrupt_callback,sleep_time=0.03)
            detector.terminate()
            interrupted=False
            i=i+1
        else: i=i;

        
    else: # 사용자가 5잔 이상 마시려고 한다면 젯봇이 좌우로 거부하는 몸짓을 하며 'stop drinking'을 출력한다.
        robot.left(0.3)
        time.sleep(0.5)
        robot.stop()
        robot.right(0.4)
        time.sleep(0.5)
        robot.stop()
        robot.left(0.3)
        time.sleep(0.5)
        robot.stop()
        robot.right(0.4)
        time.sleep(0.5)
        robot.stop()
        print('stop drinking!')
        i=i+1